This file contains various models built and scores calculated. This is one of the runs that we ran on our data. We have two other runs as well

# **Run 1:**
**Datasets used for training:**

1.   CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv
2.   CreateData_ChainOfThoughts_v4_final.csv
3.   CreateData_ChainOfThoughts_v5_final.csv

**Dataset used for testing:**

1.   CreateData_ChainOfThoughts_v6_final.csv


# **Run 2:**
**Datasets used for training:**

1.   CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv
2.   CreateData_ChainOfThoughts_v4_final.csv
3.   CreateData_ChainOfThoughts_v6_final.csv

**Dataset used for testing:**

1.   CreateData_ChainOfThoughts_v5_final.csv

# **Run 3:**

**Datasets used for training:**

1.   CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv
2.   CreateData_ChainOfThoughts_v5_final.csv
3.   CreateData_ChainOfThoughts_v6_final.csv


**Dataset used for testing:**

1. CreateData_ChainOfThoughts_v4_final.csv

Across three runs CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv is used. This contains more recipe pairs and provides good results when trained on this data

Datasets Used for Training: CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv, CreateData_ChainOfThoughts_v4_final.csv, CreateData_ChainOfThoughts_v5_final.csv

In [ ]:
#--------- Install required Packages ---------------- #
!pip install torch
!pip install -U accelerate
!pip install -U transformers

In [ ]:
#--------- Install required Packages ---------------- #
try:
  import transformers
  from sentence_transformers import SentenceTransformer
  import torch.nn as nn
except ImportError:
  !pip install -U transformers
  !pip install imageio-ffmpeg
  !pip install colabgymrender==1.0.2
  !pip install xvfbwrapper
  !pip install imageio==2.4.1
  !pip install imageio-ffmpeg
  !pip install huggingface_hub
  !pip install accelerate -U
  #!pip install transformers[torch]
  !pip install -U sentence_transformers
  !pip install dbutils
  !pip install datasets
  print("Package is installed.")
  print("Package is not installed.")

In [ ]:
#--------- Import required Packages ---------------- #
try:
  from datasets import load_dataset
except:
  !pip install datasets
  from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util, models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import InputExample

# Read custom dataset & convert to hugging face dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
Custom_Data=pd.read_csv('folder/CreateData_ChainOfThoughts_v4_final.csv')

Mounted at /content/drive


In [ ]:
Custom_Data=Custom_Data[['Recipe1','Recipe2','Similarity']]

Recipe1                        Recipe2  Similarity
0  ""Jewell Ball'S Chicken""             ""Angel Biscuits""         0.1
1  ""Jewell Ball'S Chicken""              ""Artichoke Dip""         0.2
2  ""Jewell Ball'S Chicken""                 ""Beer Bread""         0.1
3  ""Jewell Ball'S Chicken""  ""Broccoli Dip For Crackers""         0.1
4  ""Jewell Ball'S Chicken""             ""Broccoli Salad""         0.1

In [ ]:
Custom_Data_additional=pd.read_csv('folder/CreateData_ChainOfThoughts_v5_final.csv')
Custom_Data_additional=Custom_Data_additional[['Recipe1','Recipe2','Similarity']]

Recipe1                        Recipe2  Similarity
0  ""Easy Fudge""             ""Angel Biscuits""        0.05
1  ""Easy Fudge""              ""Artichoke Dip""        0.05
2  ""Easy Fudge""                 ""Beer Bread""        0.05
3  ""Easy Fudge""  ""Broccoli Dip For Crackers""        0.05
4  ""Easy Fudge""              ""Chicken Divan""        0.05

In [ ]:
Custom_Data_additional_v2=pd.read_csv('folder/CreateData_ChainOfThoughts_v3_final_With2Deciaml_Precision.csv')
Custom_Data_additional_v2=Custom_Data_additional_v2[['Recipe1','Recipe2','Similarity']]

Recipe1                 Recipe2  Similarity
0             ""7-Up Cake""            ""Corn Dip""        0.20
1        ""Angel Biscuits""       ""Sarie'S Chili""        0.20
2     ""Apple Coffee Cake""      ""Angel Biscuits""        0.60
3             ""Apple Pie""       ""Heavenly Hash""        0.30
4  ""Apples In Irish Mist""  ""Broccoli Cornbread""        0.15

In [ ]:
data = pd.concat([Custom_Data, Custom_Data_additional, Custom_Data_additional_v2])
print(data.shape)
data.head()

(7121, 3)


Recipe1                        Recipe2  Similarity
0  ""Jewell Ball'S Chicken""             ""Angel Biscuits""         0.1
1  ""Jewell Ball'S Chicken""              ""Artichoke Dip""         0.2
2  ""Jewell Ball'S Chicken""                 ""Beer Bread""         0.1
3  ""Jewell Ball'S Chicken""  ""Broccoli Dip For Crackers""         0.1
4  ""Jewell Ball'S Chicken""             ""Broccoli Salad""         0.1

# Read Recipe NLG Dataset

In [ ]:
full_dataset=pd.read_csv('folder/RecipeNLG_dataset.csv')
full_dataset = full_dataset[['title','ingredients','directions']]
full_dataset = full_dataset.reset_index()

In [ ]:
full_dataset.head()

index                  title  \
0      0    No-Bake Nut Cookies   
1      1  Jewell Ball'S Chicken   
2      2            Creamy Corn   
3      3          Chicken Funny   
4      4   Reeses Cups(Candy)     

                                         ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  
0  ["In a heavy 2-quart saucepan, mix brown sugar...  
1  ["Place chipped beef on bottom of baking dish....  
2  ["In a slow cooker, combine all ingredients. C...  
3  ["Boil and debone chicken.", "Put bite size pi...  
4  ["Combine first four ingredients and press in ...

In [ ]:
print(full_dataset.shape)
full_dataset['sentences'] = full_dataset['title']+' '+full_dataset['ingredients']+' '+full_dataset['directions']
full_dataset.head()

(2231142, 4)


index                  title  \
0      0    No-Bake Nut Cookies   
1      1  Jewell Ball'S Chicken   
2      2            Creamy Corn   
3      3          Chicken Funny   
4      4   Reeses Cups(Candy)     

                                         ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  \
0  ["In a heavy 2-quart saucepan, mix brown sugar...   
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                           sentences  
0  No-Bake Nut Cookies ["1 c. firmly packed brown...  
1  Jewell Ball'S Chicken ["1 small jar chipped be...  
2  Creamy Corn ["2 (16 oz.) pkg. frozen corn", "1...  
3  Chicken Funny ["1 large whole chicken", "2 (10...  
4  Reeses Cups(Candy)   ["1 c. peanut butter", "3...

# Operations on custom dataset

In [ ]:
data.describe()

Similarity
count  7049.000000
mean      0.233171
std       0.219412
min       0.000000
25%       0.100000
50%       0.150000
75%       0.300000
max       1.000000

In [ ]:
data['Recipe1']=data['Recipe1'].str.replace('""','')
data['Recipe2']=data['Recipe2'].str.replace('""','')
data['Recipe1']=data['Recipe1'].str.replace('""""','')
data['Recipe2']=data['Recipe2'].str.replace('""""','')
data['Recipe1']=data['Recipe1'].str.replace('"','')
data['Recipe2']=data['Recipe2'].str.replace('"','')
# Use `.loc` to address SettingWithCopyWarning
def convert_to_float(x):
  try:
    #print(float(x))
    x=float(x)
    return x
  except:
    return None

data.loc[data['Similarity'] == '', 'Similarity'] = None
data.loc[data['Similarity'] == '.', 'Similarity'] = None
data['Similarity']=data['Similarity'].apply(convert_to_float)

In [ ]:
data=data[~data.isna()]
data=data[~data['Similarity'].isna()]
data.shape

(7049, 3)

In [ ]:
data=data.groupby(['Recipe1','Recipe2']).agg({'Similarity':'mean'})
data.head()

Similarity
Recipe1            Recipe2                           
*Apricot Salad     Pat'S Soy Ginger Glaze        0.30
*Chocolate Pie*    Baked Chicken Reuben          0.01
- Almond Roca      Friendship Tea                0.20
- Angel Delights   Scalloped Wax Beans           0.15
- Banana Nut Bread Fruit Soup                    0.20

In [ ]:
data=data.reset_index()
data.head()

Recipe1                 Recipe2  Similarity
0      *Apricot Salad  Pat'S Soy Ginger Glaze        0.30
1     *Chocolate Pie*    Baked Chicken Reuben        0.01
2       - Almond Roca          Friendship Tea        0.20
3    - Angel Delights     Scalloped Wax Beans        0.15
4  - Banana Nut Bread              Fruit Soup        0.20

# Join Recipe NLG and Custom Data

In [ ]:
full_dataset['title']=full_dataset['title'].replace('"','')

In [ ]:
print(data.shape)
data=data.merge(full_dataset[['title','sentences']],how='left',left_on='Recipe1',right_on='title')
data=data.drop_duplicates(subset=['Recipe1','Recipe2'])
print(data.shape)

(5136, 3)
(5136, 5)


In [ ]:
print(data.shape)
data=data.merge(full_dataset[['title','sentences']],how='left',left_on='Recipe2',right_on='title')
data.head()
data=data.drop_duplicates(subset=['Recipe1','Recipe2'])
print(data.shape)

(5136, 5)
(5136, 7)


In [ ]:
data.head(30)

Recipe1  \
0                                  *Apricot Salad   
1                                 *Chocolate Pie*   
53                                  - Almond Roca   
147                              - Angel Delights   
148                            - Banana Nut Bread   
185               - Bee Sting Buns(Makes 16 Buns)   
187                   - Bertha'S Macaroni Delight   
191                             - Blonde Brownies   
199                         - Blueberry Dump Cake   
418                            - Blueberry Heaven   
448                         - Breakfast Casserole   
449               - Breakfast Casserole(Serves 8)   
450                          - Broccoli Casserole   
539                     - Broiled Herbed Tomatoes   
661                   - Browned Chicken And Salsa   
662             - Caramel Layer Chocolate Squares   
663                             - Cheddar Chowder   
761                                  - Cheese Dip   
1639                 - Cheese Patties With Pecans   
1697                          - Chicken Casserole   
1852                        - Chicken Inspiration   
1854                        - Chocolate Cream Log   
1879                        - Chocolate Fudge Pie   
2133              - Chocolate Peanut Butter Puffs   
2134                    - Chocolate Zucchini Cake   
2239  - Christmas All Through The House Potpourri   
2459                      - Christmas Cheer Punch   
2460                               - Congo Square   
2461                               - Cornish Hens   
2670                     - Crab Spread(Appetizer)   

                                Recipe2  Similarity          title_x  \
0                Pat'S Soy Ginger Glaze        0.30   *Apricot Salad   
1                  Baked Chicken Reuben        0.01  *Chocolate Pie*   
53                       Friendship Tea        0.20              NaN   
147                 Scalloped Wax Beans        0.15              NaN   
148                          Fruit Soup        0.20              NaN   
185                            Mac Bake        0.20              NaN   
187                 Cucumber Appetizers        0.10              NaN   
191         Parmesan Oven-Fried Chicken        0.15              NaN   
199                    Pina Colada Cake        0.70              NaN   
418                Whole Wheat Biscuits        0.20              NaN   
448             Surprise Cinnamon Rolls        0.45              NaN   
449     Crock-Pot Swedish Cabbage Rolls        0.40              NaN   
450                   Hollandaise Sauce        0.15              NaN   
539                  Oven-Fried Chicken        0.55              NaN   
661   Microwave Toasted Seasoned Pecans        0.20              NaN   
662                 Halibut Cheese Bake        0.15              NaN   
663                      Orange Sherbet        0.20              NaN   
761                         Pumpkin Pie        0.10              NaN   
1639                     Bacon Roll-Ups        0.30              NaN   
1697                  Cherry Cheese Pie        0.20              NaN   
1852                    Shrimp Toasties        0.25              NaN   
1854                        Silly Putty        0.01              NaN   
1879                    Zucchini Relish        0.10              NaN   
2133            Creamy Shrimp Appetizer        0.15              NaN   
2134                         Shortbread        0.30              NaN   
2239                Chocolate Pecan Pie        0.00              NaN   
2459  Baked Cinnamon Twists Or Crullers        0.30              NaN   
2460               Bistecca Alla Teresa        0.30              NaN   
2461                     Party Potatoes        0.20              NaN   
2670            Canadian Vegetable Soup        0.30              NaN   

                                            sentences_x  \
0     *Apricot Salad ["1 large pkg. apricot jello (6...   
1     *Chocolate Pie* ["2 c. sugar", "1/2 c. co

In [ ]:
data.isna().value_counts()

Recipe1  Recipe2  Similarity  title_x  sentences_x  title_y  sentences_y
False    False    False       False    False        False    False          4185
                                                    True     True            415
                              True     True         False    False           415
                                                    True     True            121
Name: count, dtype: int64

In [ ]:
data=data.dropna()
print(data.shape)
data=data.rename(columns={'sentences_x':'Recipe1_sentences','sentences_y':'Recipe2_sentences'})
data.head()

(4185, 7)


Recipe1                    Recipe2  Similarity  \
0                   *Apricot Salad     Pat'S Soy Ginger Glaze        0.30   
1                  *Chocolate Pie*       Baked Chicken Reuben        0.01   
16996                 1-2-3-4 Cake      Oven Baked Pork Chops        0.10   
17006     10 Minute Peanut Brittle  Old Fashioned Apple Crisp        0.20   
17014  15 Minute Lemon Chiffon Pie                 Bacon Stix        0.10   

                           title_x  \
0                   *Apricot Salad   
1                  *Chocolate Pie*   
16996                 1-2-3-4 Cake   
17006     10 Minute Peanut Brittle   
17014  15 Minute Lemon Chiffon Pie   

                                       Recipe1_sentences  \
0      *Apricot Salad ["1 large pkg. apricot jello (6...   
1      *Chocolate Pie* ["2 c. sugar", "1/2 c. cocoa",...   
16996  1-2-3-4 Cake ["1 c. butter", "2 c. sugar", "4 ...   
17006  10 Minute Peanut Brittle ["1 c. sugar", "1/2 c...   
17014  15 Minute Lemon Chiffon Pie ["1 ready crust gr...   

                         title_y  \
0         Pat'S Soy Ginger Glaze   
1           Baked Chicken Reuben   
16996      Oven Baked Pork Chops   
17006  Old Fashioned Apple Crisp   
17014                 Bacon Stix   

                                       Recipe2_sentences  
0      Pat'S Soy Ginger Glaze ["6 cloves garlic, minc...  
1      Baked Chicken Reuben ["4 whole chicken breasts...  
16996  Oven Baked Pork Chops ["1/2 stick butter", "1 ...  
17006  Old Fashioned Apple Crisp ["1 can (20 oz.) Tha...  
17014  Bacon Stix ["10 thin bread sticks (any flavor)...

# Convert to Huggingface dataset

In [ ]:
from datasets import load_dataset,Dataset
dataset=Dataset.from_pandas(data)

In [ ]:
datasetB=dataset
print(datasetB)

Dataset({
    features: ['Recipe1', 'Recipe2', 'Similarity', 'title_x', 'Recipe1_sentences', 'title_y', 'Recipe2_sentences', '__index_level_0__'],
    num_rows: 4185
})


In [ ]:
print(f"- The dataset has {datasetB.num_rows} examples.")
print(f"- Each example is a {type(datasetB[0])} with a {type(datasetB[0])} as value.")
print(f"- Examples look like this: {datasetB[0]}")

- The dataset has 4185 examples.
- Each example is a <class 'dict'> with a <class 'dict'> as value.
- Examples look like this: {'Recipe1': '*Apricot Salad', 'Recipe2': "Pat'S Soy Ginger Glaze", 'Similarity': 0.3, 'title_x': '*Apricot Salad', 'Recipe1_sentences': '*Apricot Salad ["1 large pkg. apricot jello (6 oz.)", "2/3 c. sugar", "1 (8 oz.) pkg. cream cheese", "2 jars Junior baby food apricots", "1 (No. 2) can crushed pineapple (20 oz.)", "1 tall can evaporated milk, chilled and whipped", "2/3 c. water"] ["Dissolve jello in boiling water; add sugar.", "Blend in the cream cheese with a fork.", "Add can of crushed pineapple and apricot baby food.", "Refrigerate.", "Chill can of milk.", "When jello starts to set, take milk out of the refrigerator and beat until stiff.", "Fold partially set jello into cream.", "Chopped nuts can be used for garnish.", "This is a large salad; could fill a 2-quart mold or 2 (1-quart) molds.", "Serves about 12."]', 'title_y': "Pat'S Soy Ginger Glaze", 'Recip

In [ ]:
datasetB.features

{'Recipe1': Value(dtype='string', id=None),
 'Recipe2': Value(dtype='string', id=None),
 'Similarity': Value(dtype='float64', id=None),
 'title_x': Value(dtype='string', id=None),
 'Recipe1_sentences': Value(dtype='string', id=None),
 'title_y': Value(dtype='string', id=None),
 'Recipe2_sentences': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

# Fine Tuned BERT with 5 EPOCs

In [ ]:
#--------- Load Model ------------------- #
modelB = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#----------- Get training data -------------- #
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
#----------- Define Model Parameters -------------- #
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 5#10
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
#----------- Fit the model -------------- #
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
#----------- Save model into folder-------------- #
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_fine_tuned_bert_5_EPOC.pt')

# Fine Tuned BERT with 10 EPOC

In [ ]:
#--------- Define model for 10 EPOC. Note that model is already trained on 5 EPOCs till now.
#--------- So, Just adding additional 5 epocs would give a trained model with 10 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 5 epocs model as modelB and start training
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 5
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
#----------- Fit the model -------------- #
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
#----------- Save model into folder-------------- #
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_fine_tuned_bert_10_EPOC.pt')

# Fine Tuned BERT with 20 EPOC

In [ ]:
#--------- Define model for 20 EPOC. Note that model is already trained on 10 EPOCs till now.
#--------- So, Just adding additional 10 epocs would give a trained model with 20 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 10 epocs model as modelB and start training

from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 10
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_fine_tuned_bert_20_EPOC.pt')

# Fine Tuned BERT with 40 EPOC

In [ ]:
#--------- Define model for 40 EPOC. Note that model is already trained on 20 EPOCs till now.
#--------- So, Just adding additional 20 epocs would give a trained model with 40 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 20 epocs model as modelB and start training

from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 20
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_fine_tuned_bert_40_EPOC.pt')

# Few Layers BERT V1

In [ ]:
#------- Defining few layers BERT model. We are loading a new model into modelB ---------- #
del modelB

modelB = SentenceTransformer('bert-base-nli-mean-tokens')

# Freeze all layers first
for param in modelB[0].auto_model.parameters():
    param.requires_grad = False

# Unfreeze the last two transformer layers
num_layers = len(modelB[0].auto_model.encoder.layer)  # Total number of transformer layers
for i in range(num_layers - 2, num_layers):
    for param in modelB[0].auto_model.encoder.layer[i].parameters():
        param.requires_grad = True

# Print out the frozen status of each layer
for i, layer in enumerate(modelB[0].auto_model.encoder.layer):
    print(f"Layer {i}, Frozen: {not any(param.requires_grad for param in layer.parameters())}")
    print(layer)

In [ ]:
for p in modelB[1].parameters():
  print(p)

In [ ]:
modelB[1].parameters()

In [ ]:
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
len(train_examplesB)

In [ ]:
EPOCS = 20

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32#16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = EPOCS
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_few_layers_bert_v1.pt')

# Few Layers BERT V2

In [ ]:
EPOCS = 20

In [ ]:
#------- Defining few layers BERT model. We are loading a new model into modelB ---------- #
del modelB
modelB = SentenceTransformer('bert-base-nli-mean-tokens')
 #('embedding-data/distilroberta-base-sentence-transformer')

# Freeze all layers first
for param in modelB[0].auto_model.parameters():
    param.requires_grad = False

# Unfreeze the last two transformer layers
num_layers = len(modelB[0].auto_model.encoder.layer)  # Total number of transformer layers
for i in range(num_layers - 1, num_layers):
    for param in modelB[0].auto_model.encoder.layer[i].parameters():
        param.requires_grad = True

# Print out the frozen status of each layer
for i, layer in enumerate(modelB[0].auto_model.encoder.layer):
    print(f"Layer {i}, Frozen: {not any(param.requires_grad for param in layer.parameters())}")
    print(layer)

In [ ]:
for p in modelB[0].auto_model.pooler.parameters():
    print('froozen', not p.requires_grad)
for p in modelB[0].auto_model.pooler.parameters():
    p.requires_grad=True
for p in modelB[0].auto_model.pooler.parameters():
    print('froozen', not p.requires_grad)

In [ ]:
for p in modelB[1].parameters():
  print(p)

In [ ]:
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
len(train_examplesB)

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32#16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = EPOCS
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
warmup_stepsB

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_few_layers_bert_v2.pt')

# Fine tuning ROBERTA - 5 EPOC

In [ ]:
#------- Defining ROBERTA. We are loading a new model into modelB ---------- #
del modelB
modelB = SentenceTransformer('distilbert/distilroberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 5#10
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
warmup_stepsB

131.0

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_roberta_5_EPOC.pt')

# Fine tuning ROBERTA - 10 EPOC

In [ ]:
#--------- Define model for 10 EPOC. Note that model is already trained on 5 EPOCs till now.
#--------- So, Just adding additional 5 epocs would give a trained model with 10 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 5 epocs model as modelB and start training

from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 5
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_roberta_10_EPOC.pt')

# Fine tuning ROBERTA - 20 EPOC

In [ ]:
#--------- Define model for 20 EPOC. Note that model is already trained on 10 EPOCs till now.
#--------- So, Just adding additional 10 epocs would give a trained model with 20 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 10 epocs model as modelB and start training

from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 10
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_roberta_20_EPOC.pt')

# Fine tuning ROBERTA - 50 EPOC

In [ ]:
#--------- Define model for 50 EPOC. Note that model is already trained on 20 EPOCs till now.
#--------- So, Just adding additional 30 epocs would give a trained model with 50 Epocs
#--------- If your session gets disconnected and you have saved the previous model, load 20 epocs model as modelB and start training

from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=modelB)
num_epochsB = 30
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(modelB.state_dict(), 'folder/Jeevan_cv_run1_roberta_50_EPOC.pt')

# Add New Layer BERT - 30 EPOCs

In [ ]:
EPOC = 30

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer, models

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
class CustomModel(nn.Module):
    def __init__(self,sentence_bert):
        self.device='cuda'
        super(CustomModel, self).__init__()
        self.tokenize1 = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer = sentence_bert[0].auto_model.to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
        """self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )"""
        self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),  # ReLU activation function
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )
        # Freeze all the parameters in the BERT model
        for param in self.transformer.parameters():
            param.requires_grad = False
        import torch.nn.init as init
        # Initialize weights using He initialization suitable for ReLU activations
        """for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)"""
        # Initialize weights using He initialization suitable for ReLU activations
        for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                # He initialization for weights
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                # Zero initialization for biases (if applicable)
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)
   # def tokenize(self, text):
    #    # Implement your tokenization logic
     #   return self.tokenize1(text,return_tensors='pt')

    def tokenize(self, text):
        return self.tokenize1(text, return_tensors='pt', padding=True, truncation=True, max_length=512)


    def forward(self, tok_text):
        #tok_text = self.tokenizer(tok_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        #print('&'*10,tok_text['input_ids'],type(tok_text['input_ids']))
        outputs = self.transformer(**tok_text)#input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        custom_output = self.custom_layers(sequence_output)

        pool_input = {
        'token_embeddings': custom_output,  # Assuming this is [batch_size, seq_len, hidden_size]
        'attention_mask': tok_text['attention_mask']
        }

        return pool_input
sentence_bert = SentenceTransformer('bert-base-nli-mean-tokens')

# Create an instance of your custom model
custom_model = CustomModel(sentence_bert)

# Since SentenceTransformer expects a model that outputs a single embedding vector per input,
# you should define a pooling layer if the output is a sequence of token embeddings.
pooling_layer = models.Pooling(custom_model.transformer.config.hidden_size)
pooling_layer = sentence_bert[1].to('cuda')

# Create a Sentence Transformer model with custom processing
model = SentenceTransformer(modules=[custom_model, pooling_layer])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pooling_layer

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})

In [ ]:
# To use the model, you need to preprocess the text outside the SentenceTransformer model call:
#example_text = "Here is an example sentence to demonstrate the model's capabilities."
model_output = model.encode('example_text')  # Directly use custom_model for text inputs
print(model_output)
#pooling_layer(model_output)

[ 1.5186617  -0.67785454  0.05808409  1.7885411   1.0431527  -0.4570352
 -0.41970205 -0.6610222   1.5439942   0.01230109 -0.67785454 -0.67785454
 -0.50208336  0.13989368 -0.51289684  1.6817226  -0.596063   -0.12635806
  1.2284262  -0.67785454  1.7796195  -0.15734233  0.27104178  0.10664403
  2.9809875  -0.5919846  -0.06123973 -0.67785454  0.4878363   0.7756364
  1.2325248   0.13582292 -0.67785454 -0.67785454 -0.65470785 -0.5808606
  0.5465387   0.4752794  -0.42659912 -0.67785454  0.24694785  0.8122021
 -0.27275413 -0.4917714  -0.48523855  0.30212986 -0.5521916  -0.67785454
 -0.67785454 -0.67785454 -0.67785454 -0.47616902 -0.67785454  3.089727
 -0.56287366 -0.59025913 -0.67785454 -0.67785454  1.7375002  -0.38594016
 -0.67785454 -0.6056648  -0.67785454 -0.67785454  0.38482395 -0.67785454
  0.3876658   0.03306852 -0.67785454 -0.43450737 -0.51429474 -0.63925433
 -0.67785454 -0.520645    1.2362268  -0.47969657 -0.6726165  -0.5993806
 -0.67785454  2.1074505  -0.67785454 -0.01849654  0.427750

In [ ]:
custom_model.transformer.config.hidden_size

768

In [ ]:
for i, layer in enumerate(model[0].transformer.encoder.layer):
    layer_name = layer.__class__.__name__  # Gets the class name of the layer
    frozen = not any(param.requires_grad for param in layer.parameters())
    print(f"Layer {i} ({layer_name}), Frozen: {frozen}")

Layer 0 (BertLayer), Frozen: True
Layer 1 (BertLayer), Frozen: True
Layer 2 (BertLayer), Frozen: True
Layer 3 (BertLayer), Frozen: True
Layer 4 (BertLayer), Frozen: True
Layer 5 (BertLayer), Frozen: True
Layer 6 (BertLayer), Frozen: True
Layer 7 (BertLayer), Frozen: True
Layer 8 (BertLayer), Frozen: True
Layer 9 (BertLayer), Frozen: True
Layer 10 (BertLayer), Frozen: True
Layer 11 (BertLayer), Frozen: True


In [ ]:
for param in custom_model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [ ]:
model_add_new_layer=model

In [ ]:
model_add_new_layer

SentenceTransformer(
  (0): CustomModel(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=

In [ ]:
model_add_new_layer[0]

CustomModel(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
len(train_examplesB)

4185

In [ ]:
EPOC

30

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32#16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=model_add_new_layer)
num_epochsB = EPOC#10
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
warmup_stepsB

393.0

In [ ]:
model_add_new_layer

SentenceTransformer(
  (0): CustomModel(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=

In [ ]:
model_add_new_layer.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(model_add_new_layer.state_dict(), 'folder/Jeevan_cv_run1_add_new_layer_bert_30_EPOC.pt')

# Add New Layer BERT - 50 EPOCs

In [ ]:
#--------- Training on the previous model with additional 30 EPOCs --------------- #

from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32#16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=model_add_new_layer)
num_epochsB = 20
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
model_add_new_layer.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(model_add_new_layer.state_dict(), 'folder/Jeevan_cv_run1_add_new_layer_bert_50_EPOC.pt')

# Add New Layer BERT - 100 EPOCs

In this case, we are loading Jeevan_cv_run1_add_new_layer_bert_50_EPOC.pt and training on additional 50 EPOCs

In [ ]:
from sentence_transformers import InputExample
train_examplesB = []
# train_dataB = datasetB['train']
# n_examples = datasetB['train'].num_rows//2

for example in datasetB:

  if float(example['Similarity'])<0 or float(example['Similarity'])>1 :
        # Skip examples without a valid label
        continue

  train_examplesB.append(InputExample(texts=[example['Recipe1_sentences'], example['Recipe2_sentences']],label=float(example['Similarity'])))

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer, models

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
class CustomModel(nn.Module):
    def __init__(self,sentence_bert):
        self.device='cuda'
        super(CustomModel, self).__init__()
        self.tokenize1 = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer = sentence_bert[0].auto_model.to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
        """self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )"""
        self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),  # ReLU activation function
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )
        # Freeze all the parameters in the BERT model
        for param in self.transformer.parameters():
            param.requires_grad = False
        import torch.nn.init as init
        # Initialize weights using He initialization suitable for ReLU activations
        """for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)"""
        # Initialize weights using He initialization suitable for ReLU activations
        for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                # He initialization for weights
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                # Zero initialization for biases (if applicable)
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)
   # def tokenize(self, text):
    #    # Implement your tokenization logic
     #   return self.tokenize1(text,return_tensors='pt')

    def tokenize(self, text):
        return self.tokenize1(text, return_tensors='pt', padding=True, truncation=True, max_length=512)


    def forward(self, tok_text):
        #tok_text = self.tokenizer(tok_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        #print('&'*10,tok_text['input_ids'],type(tok_text['input_ids']))
        outputs = self.transformer(**tok_text)#input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        custom_output = self.custom_layers(sequence_output)

        pool_input = {
        'token_embeddings': custom_output,  # Assuming this is [batch_size, seq_len, hidden_size]
        'attention_mask': tok_text['attention_mask']
        }

        return pool_input
sentence_bert = SentenceTransformer('bert-base-nli-mean-tokens')

# Create an instance of your custom model
custom_model = CustomModel(sentence_bert)

# Since SentenceTransformer expects a model that outputs a single embedding vector per input,
# you should define a pooling layer if the output is a sequence of token embeddings.
pooling_layer = models.Pooling(custom_model.transformer.config.hidden_size)
pooling_layer = sentence_bert[1].to('cuda')

# Create a Sentence Transformer model with custom processing
model_add_new_layer = SentenceTransformer(modules=[custom_model, pooling_layer])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_add_new_layer.load_state_dict(torch.load('folder/Jeevan_cv_run1_add_new_layer_bert_50_EPOC.pt'))

<All keys matched successfully>

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32#16#64
                               )
train_lossB = losses.CosineSimilarityLoss(model=model_add_new_layer)
num_epochsB = 50
warmup_stepsB = len(train_dataloaderB) * num_epochsB * 0.1 #10% of train data

In [ ]:
model_add_new_layer.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
import torch
torch.save(model_add_new_layer.state_dict(), 'folder/Jeevan_cv_run1_add_new_layer_bert_100_EPOC.pt')

# Calculating Model Scores

In [ ]:
!pip install -q pandas
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q nltk

In [ ]:
#in google colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


Reading RECIPE NLG dataset

In [ ]:
import pandas as pd
import pickle
import numpy as np
#import nltk
full_data=pd.read_csv('folder/RecipeNLG_dataset.csv')

Reading Cross Validation / Testing dataset

In [ ]:
sample_data = 'folder/CreateData_ChainOfThoughts_v6_final.csv'
sample_data=pd.read_csv(sample_data)
sample_data.head()

In [ ]:
sample_data.groupby('Recipe1').agg(count=('Recipe2', 'count'))

In [ ]:
sample_data = sample_data[["Recipe1", "Recipe2", "Similarity"]]

In [ ]:
sample_data=sample_data.rename(columns={'Recipe1':'title_Recipe1',"Recipe2":"title_Recipe2"})
sample_data.shape

In [ ]:
# First, create a modified version of both columns without quotes
sample_data['title_Recipe1_mod'] = sample_data['title_Recipe1'].str.replace('"', '')
full_data['title_mod'] = full_data['title'].str.replace('"', '')
sample_data['title_Recipe2_mod'] = sample_data['title_Recipe2'].str.replace('"', '')

In [ ]:
sample_data.head()

In [ ]:
import pandas as pd

# Assuming sample_data is your DataFrame
# Define the conversion function
def convrt(x):
    try:
        return float(x)
    except:
        return None

# Apply the conversion function to the 'Similarity' column
sample_data['Similarity'] = sample_data['Similarity'].apply(convrt)

# Filter rows where 'Similarity' is not None (i.e., conversion was successful)
sample_data = sample_data.dropna(subset=['Similarity'])

In [ ]:
sample_data=sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod'])
sample_data.shape

In [ ]:

# Create a set of unique modified titles from full_data for efficient lookup
full_data_titles_set = set(full_data['title_mod'].unique())

# Use the set to filter sample_data
sample_data = sample_data[sample_data['title_Recipe1_mod'].isin(full_data_titles_set)]
sample_data = sample_data[sample_data['title_Recipe2_mod'].isin(full_data_titles_set)]

# Now filtered_sample_data contains rows from sample_data where title_Recipe1 (quotes removed)
# does not exactly match any title in full_data (quotes removed)

In [ ]:
print(sample_data.shape)
sample_data.head()

In [ ]:
sample_data=sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod'])
sample_data.shape

# TFIDF Calcaution on sample_data
sample_data is the cross validation dataset

In [ ]:
sample_data=sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod'])
print(sample_data.shape)

In [ ]:
Recipes_list=pd.concat([sample_data['title_Recipe1_mod'],sample_data['title_Recipe2_mod']])
print(Recipes_list.shape)
print(sample_data.shape)
sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod']).shape

In [ ]:
Recipes_list=Recipes_list.unique()
len(Recipes_list)

In [ ]:
sample_data

In [ ]:
data=full_data[full_data['title_mod'].isin(Recipes_list)]

In [ ]:
data=data.drop_duplicates(subset=['title_mod'])

In [ ]:
data['title_mod'].drop_duplicates().shape

In [ ]:
full_recipe_NER = dict(zip(data['title_mod'], data['NER']))
full_recipe_NER

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
# # Sample RecipeNLG dataset with recipe ingredients (for demonstration purposes)
# sample_recipe_ingredients = {
# 'Spaghetti Carbonara': 'spaghetti, eggs, pancetta, pecorino cheese, salt, black pepper',
# 'Chicken Curry': 'chicken, curry powder, garlic, onion, coconut milk, salt',
# 'Beef Stroganoff': 'beef, mushrooms, onion, garlic, sour cream, salt, black pepper',
# 'Vegetable Stir Fry': 'broccoli, bell peppers, carrots, garlic, soy sauce, salt',
# 'Fish Tacos': 'fish, tortillas, cabbage, lime, mayonnaise, salt',
# 'Pumpkin Soup': 'pumpkin, onion, garlic, vegetable broth, salt, black pepper',
# 'Chocolate Cake': 'flour, cocoa powder, sugar, eggs, butter, baking powder',
# 'Apple Pie': 'apples, flour, sugar, butter, cinnamon, salt',
# 'Caesar Salad': 'lettuce, croutons, parmesan cheese, caesar dressing, black pepper',
# 'Grilled Cheese': 'bread, cheese, butter, salt'
# }
# Display sample recipe ingredients
sample_recipe_NER=full_recipe_NER

In [ ]:
# Initialize Stemmer and Lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Function to preprocess, stem and lemmatize ingredients
def preprocess_ingredients(ingredients):
    # Tokenize and remove commas
    tokens = ingredients.replace(',', '').split()
    # Apply stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # Apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    return ' '.join(lemmatized_tokens)
# Apply preprocessing, stemming and lemmatization to the sample ingredients
processed_NER = {title: preprocess_ingredients(NER) for title, NER in sample_recipe_NER.items()}
# Display processed ingredients

In [ ]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the processed ingredients to get the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(list(processed_NER.values()))
# Calculate cosine similarity between the TF-IDF vectors
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# Create a DataFrame to store the similarity values
similarity_df = pd.DataFrame(cosine_sim, index=processed_NER.keys(), columns=processed_NER.keys())
# Display the similarity DataFrame
similarity_df

In [ ]:
def lookup_similarity(row, similarity_matrix):
    # Get the titles from the row
    title1 = row['title_Recipe1_mod']
    title2 = row['title_Recipe2_mod']

    # Check if both titles are in the similarity matrix index and columns
    if title1 in similarity_matrix.index and title2 in similarity_matrix.columns:
        # Return the similarity score
        return similarity_matrix.loc[title1, title2]
    else:
        # Return None if the titles aren't found
        return None

# Apply the function to each row of sample_data
sample_data['tfidf_similarity'] = sample_data.apply(lambda row: lookup_similarity(row, similarity_df), axis=1)


In [ ]:
print(sample_data.shape)
sample_data

# Calcuating Similarity Scores using BERT

In [ ]:
sample_data.head()

Importing .pkl file for full dataset

In [ ]:
import pickle
import pandas as pd
sentence_vecs=pd.read_pickle("folder/fulldataset_sentence_vecs_v1.pkl")
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd
import numpy as np

Filter_index=data[data['title'].isin(similarity_df.columns)].index.values
#Filter_index=similarity_df['title']
Filter_index=Filter_index
print(Filter_index.shape)
Filter_index

In [ ]:
predicted_similarity_matrix_filter = cosine_similarity(sentence_vecs[Filter_index, :])
predicted_similarity_matrix_filter

In [ ]:
# Assume cs is your cosine similarity matrix and has the same number of rows as sentence_vecs
cs_df = pd.DataFrame(predicted_similarity_matrix_filter)

In [ ]:
cs_df = pd.DataFrame(predicted_similarity_matrix_filter)

In [ ]:
cs_df.index=data[full_data['title'].isin(similarity_df.columns)]['title']
cs_df.columns=cs_df.index

In [ ]:
def lookup_bert_similarity(row, similarity_matrix):
    # Get the titles from the row
    title1 = row['title_Recipe1_mod']
    title2 = row['title_Recipe2_mod']

    # Check if both titles are in the similarity matrix index and columns
    if title1 in similarity_matrix.index and title2 in similarity_matrix.columns:
        # Return the similarity score
        try:
            return float(similarity_matrix.loc[title1, title2])
        except:
            try:
                return float(similarity_matrix.loc[title1, title2].mean())
            except:
                try:
                    return float(similarity_matrix.loc[title1, title2].mean().mean())
                except:
                    print('='*10,similarity_matrix.loc[title1, title2])
                    return None
    else:
        # Return None if the titles aren't found
        return None

# Apply the function to each row of sample_data
sample_data['Bert_similarity'] = sample_data.apply(lambda row: lookup_bert_similarity(row, cs_df), axis=1)


In [ ]:
sample_data['Bert_similarity'].isna().value_counts()

In [ ]:
sample_data.head()

# Calculating similarity scores using Fine Tuned BERT - 5 EPOC

In [ ]:
sample_data.head()

title_Recipe1       title_Recipe2  Similarity title_Recipe1_mod  \
0  ""Chicken Stew""  ""Angel Biscuits""        0.05      Chicken Stew   
1  ""Chicken Stew""      ""Beer Bread""        0.05      Chicken Stew   
2  ""Chicken Stew""   ""Chicken Divan""        0.85      Chicken Stew   
3  ""Chicken Stew""     ""Chicken Ole""        0.80      Chicken Stew   
4  ""Chicken Stew""    ""Chicken Stew""        1.00      Chicken Stew   

  title_Recipe2_mod  tfidf_similarity  Bert_similarity         title_x  \
0    Angel Biscuits          0.024506         0.718295  Angel Biscuits   
1        Beer Bread          0.048481         0.753868      Beer Bread   
2     Chicken Divan          0.043482         0.913871   Chicken Divan   
3       Chicken Ole          0.105818         0.895192     Chicken Ole   
4      Chicken Stew          1.000000         1.000000    Chicken Stew   

                                   Recipe1_sentences       title_y  ...  \
0  Angel Biscuits ["5 c. flour", "3 Tbsp. sugar",...  Chicken Stew  ...   
1  Beer Bread ["3 c. self rising flour", "1 - 12 ...  Chicken Stew  ...   
2  Chicken Divan ["1/4 c. margarine", "1/4 c. cho...  Chicken Stew  ...   
3  Chicken Ole ["4 chicken breasts, cooked", "1 c...  Chicken Stew  ...   
4  Chicken Stew ["3 lb. chicken, boiled", "4 medi...  Chicken Stew  ...   

  Few_Layers_Bert_Similarity_v1  Few_Layers_Bert_Similarity_v2  \
0                      0.156657                       0.154250   
1                      0.095093                       0.083473   
2                      0.777404                       0.771455   
3                      0.846453                       0.840743   
4                      1.000000                       1.000000   

   Fine_Tuned_Bert_Similarity_10_epoc  Fine_Tuned_Bert_Similarity_20_epoc  \
0                            0.147811                            0.141874   
1                            0.108890                            0.084739   
2                            0.820424                            0.829184   
3                            0.865740                            0.852861   
4                            1.000000                            1.000000   

   Robert_Similarity_5_epoc  Robert_Similarity_10_epoc  \
0                  0.131144                   0.129336   
1                  0.124134                   0.090098   
2                  0.808497                   0.840993   
3                  0.870588                   0.864439   
4                  1.000000                   1.000000   

   Robert_Similarity_20_epoc  Robert_Similarity_30_epoc  \
0                   0.137537                   0.132106   
1                   0.089476                   0.083370   
2                   0.844403                   0.838664   
3                   0.864118                   0.848837   
4                   1.000000                   1.000000   

   RecipeBert_finetuning_v3  Fine_Tuned_Bert_Similarity_40_epoc  
0                  0.138439                            0.155239  
1                  0.099758                            0.082167  
2                  0.877858                            0.851222  
3                  0.901725                            0.845041  
4                  1.000000                            1.000000  

[5 rows x 22 columns]

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_fine_tuned_bert_5_EPOC.pt'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# #in google colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

full_data=pd.read_csv('folder/RecipeNLG_dataset.csv')
sample_data.head()

In [ ]:
print(full_data.shape)
full_data.head()

In [ ]:
full_data['sentences'] = full_data['title']+' '+full_data['ingredients']+' '+full_data['directions']
full_data.head()

In [ ]:
full_data['title']=full_data['title'].replace('"','')
print(sample_data.shape)
sample_data=sample_data.merge(full_data[['title','sentences']],how='left',left_on='title_Recipe2_mod',right_on='title')
sample_data=sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod'])
print(sample_data.shape)
sample_data=sample_data.merge(full_data[['title','sentences']],how='left',left_on='title_Recipe1_mod',right_on='title')
sample_data=sample_data.drop_duplicates(subset=['title_Recipe1_mod','title_Recipe2_mod'])
print(sample_data.shape)

In [ ]:
sample_data=sample_data.rename(columns={'sentences_x':'Recipe1_sentences','sentences_y':'Recipe2_sentences'})

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Fine_Tuned_Bert_Similarity']=None


(1855, 22)
(1855, 22)


In [ ]:

from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Fine_Tuned_Bert_Similarity']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

In [ ]:
sample_data.head()

title_Recipe1       title_Recipe2  Similarity title_Recipe1_mod  \
0  ""Chicken Stew""  ""Angel Biscuits""        0.05      Chicken Stew   
1  ""Chicken Stew""      ""Beer Bread""        0.05      Chicken Stew   
2  ""Chicken Stew""   ""Chicken Divan""        0.85      Chicken Stew   
3  ""Chicken Stew""     ""Chicken Ole""        0.80      Chicken Stew   
4  ""Chicken Stew""    ""Chicken Stew""        1.00      Chicken Stew   

  title_Recipe2_mod  tfidf_similarity  Bert_similarity         title_x  \
0    Angel Biscuits          0.024506         0.718295  Angel Biscuits   
1        Beer Bread          0.048481         0.753868      Beer Bread   
2     Chicken Divan          0.043482         0.913871   Chicken Divan   
3       Chicken Ole          0.105818         0.895192     Chicken Ole   
4      Chicken Stew          1.000000         1.000000    Chicken Stew   

                                   Recipe1_sentences       title_y  ...  \
0  Angel Biscuits ["5 c. flour", "3 Tbsp. sugar",...  Chicken Stew  ...   
1  Beer Bread ["3 c. self rising flour", "1 - 12 ...  Chicken Stew  ...   
2  Chicken Divan ["1/4 c. margarine", "1/4 c. cho...  Chicken Stew  ...   
3  Chicken Ole ["4 chicken breasts, cooked", "1 c...  Chicken Stew  ...   
4  Chicken Stew ["3 lb. chicken, boiled", "4 medi...  Chicken Stew  ...   

  Few_Layers_Bert_Similarity_v1  Few_Layers_Bert_Similarity_v2  \
0                      0.156657                       0.154250   
1                      0.095093                       0.083473   
2                      0.777404                       0.771455   
3                      0.846453                       0.840743   
4                      1.000000                       1.000000   

   Fine_Tuned_Bert_Similarity_10_epoc  Fine_Tuned_Bert_Similarity_20_epoc  \
0                            0.147811                            0.141874   
1                            0.108890                            0.084739   
2                            0.820424                            0.829184   
3                            0.865740                            0.852861   
4                            1.000000                            1.000000   

   Robert_Similarity_5_epoc  Robert_Similarity_10_epoc  \
0                  0.131144                   0.129336   
1                  0.124134                   0.090098   
2                  0.808497                   0.840993   
3                  0.870588                   0.864439   
4                  1.000000                   1.000000   

   Robert_Similarity_20_epoc  Robert_Similarity_30_epoc  \
0                   0.137537                   0.132106   
1                   0.089476                   0.083370   
2                   0.844403                   0.838664   
3                   0.864118                   0.848837   
4                   1.000000                   1.000000   

   RecipeBert_finetuning_v3  Fine_Tuned_Bert_Similarity_40_epoc  
0                  0.138439                            0.155239  
1                  0.099758                            0.082167  
2                  0.877858                            0.851222  
3                  0.901725                            0.845041  
4                  1.000000                            1.000000  

[5 rows x 22 columns]

# Calculating similarity scores using Fine Tuned BERT - 10 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_fine_tuned_bert_10_EPOC.pt', map_location=torch.device('cpu')))

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np


print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Fine_Tuned_Bert_Similarity_10_epoc']=None

from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Fine_Tuned_Bert_Similarity_10_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


# Calculating similarity scores using Fine Tuned BERT - 20 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_fine_tuned_bert_20_EPOC.pt', map_location=torch.device('cpu')))

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Fine_Tuned_Bert_Similarity_20_epoc']=None

from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Fine_Tuned_Bert_Similarity_20_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


# Calculating similarity scores using Fine Tuned BERT - 40 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_fine_tuned_bert_40_EPOC.pt',
                                                  map_location=torch.device('cpu')))

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Fine_Tuned_Bert_Similarity_40_epoc']=None

from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Fine_Tuned_Bert_Similarity_40_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


In [ ]:
sample_data.head()

title_Recipe1       title_Recipe2  Similarity title_Recipe1_mod  \
0  ""Chicken Stew""  ""Angel Biscuits""        0.05      Chicken Stew   
1  ""Chicken Stew""      ""Beer Bread""        0.05      Chicken Stew   
2  ""Chicken Stew""   ""Chicken Divan""        0.85      Chicken Stew   
3  ""Chicken Stew""     ""Chicken Ole""        0.80      Chicken Stew   
4  ""Chicken Stew""    ""Chicken Stew""        1.00      Chicken Stew   

  title_Recipe2_mod  tfidf_similarity  Bert_similarity         title_x  \
0    Angel Biscuits          0.024506         0.718295  Angel Biscuits   
1        Beer Bread          0.048481         0.753868      Beer Bread   
2     Chicken Divan          0.043482         0.913871   Chicken Divan   
3       Chicken Ole          0.105818         0.895192     Chicken Ole   
4      Chicken Stew          1.000000         1.000000    Chicken Stew   

                                   Recipe1_sentences       title_y  ...  \
0  Angel Biscuits ["5 c. flour", "3 Tbsp. sugar",...  Chicken Stew  ...   
1  Beer Bread ["3 c. self rising flour", "1 - 12 ...  Chicken Stew  ...   
2  Chicken Divan ["1/4 c. margarine", "1/4 c. cho...  Chicken Stew  ...   
3  Chicken Ole ["4 chicken breasts, cooked", "1 c...  Chicken Stew  ...   
4  Chicken Stew ["3 lb. chicken, boiled", "4 medi...  Chicken Stew  ...   

  Few_Layers_Bert_Similarity_v1  Few_Layers_Bert_Similarity_v2  \
0                      0.156657                       0.154250   
1                      0.095093                       0.083473   
2                      0.777404                       0.771455   
3                      0.846453                       0.840743   
4                      1.000000                       1.000000   

   Fine_Tuned_Bert_Similarity_10_epoc  Fine_Tuned_Bert_Similarity_20_epoc  \
0                            0.147811                            0.141874   
1                            0.108890                            0.084739   
2                            0.820424                            0.829184   
3                            0.865740                            0.852861   
4                            1.000000                            1.000000   

   Robert_Similarity_5_epoc  Robert_Similarity_10_epoc  \
0                  0.131144                   0.129336   
1                  0.124134                   0.090098   
2                  0.808497                   0.840993   
3                  0.870588                   0.864439   
4                  1.000000                   1.000000   

   Robert_Similarity_20_epoc  Robert_Similarity_30_epoc  \
0                   0.137537                   0.132106   
1                   0.089476                   0.083370   
2                   0.844403                   0.838664   
3                   0.864118                   0.848837   
4                   1.000000                   1.000000   

   RecipeBert_finetuning_v3  Fine_Tuned_Bert_Similarity_40_epoc  
0                  0.138439                            0.155239  
1                  0.099758                            0.082167  
2                  0.877858                            0.851222  
3                  0.901725                            0.845041  
4                  1.000000                            1.000000  

[5 rows x 22 columns]

# Calculating similarity scores using Few Layers BERT v1

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Few_Layers_Bert = SentenceTransformer('bert-base-nli-mean-tokens')
Few_Layers_Bert = torch.load('folder/Jeevan_cv_run1_few_layers_bert_v1.pt',
                                    map_location=torch.device('cuda'))

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Few_Layers_Bert.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Few_Layers_Bert.encode(list_recipe2, convert_to_tensor=True)
sample_data['Few_Layers_Bert_Similarity_v1']=None

In [ ]:

from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Few_Layers_Bert_Similarity_v1']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

In [ ]:
sample_data.head()

# Calculating similarity scores using Few Layers BERT v2

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Few_Layers_Bert = SentenceTransformer('bert-base-nli-mean-tokens')
Few_Layers_Bert = torch.load('folder/Jeevan_cv_run1_few_layers_bert_v2.pt',
                             map_location=torch.device('cuda'))

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Few_Layers_Bert.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Few_Layers_Bert.encode(list_recipe2, convert_to_tensor=True)
sample_data['Few_Layers_Bert_Similarity_v2']=None

In [ ]:
from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Few_Layers_Bert_Similarity_v2']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

# Calculating similarity scores using ROBERTA 5 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('distilbert/distilroberta-base')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_roberta_5_EPOC.pt',
                                                  map_location=torch.device('cpu')))

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Robert_Similarity_5_epoc']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Robert_Similarity_5_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


# Calculating similarity scores usig ROBERTA 10 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('distilbert/distilroberta-base')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_roberta_10_EPOC.pt',
                                                  map_location=torch.device('cpu')))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Robert_Similarity_10_epoc']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Robert_Similarity_10_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


# Calculating similarity scores using ROBERTA 20 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('distilbert/distilroberta-base')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_roberta_20_EPOC.pt',
                                                  map_location=torch.device('cpu')))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Robert_Similarity_20_epoc']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Robert_Similarity_20_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


In [ ]:
sample_data.head()

# Calculating similarity scores using ROBERTA 50 EPOC

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Bert_Fined_Tuned_Model = SentenceTransformer('distilbert/distilroberta-base')
Bert_Fined_Tuned_Model.load_state_dict(torch.load('folder/Jeevan_cv_run1_roberta_50_EPOC.pt',
                                                  map_location=torch.device('cpu')))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Bert_Fined_Tuned_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Bert_Fined_Tuned_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Robert_Similarity_50_epoc']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Robert_Similarity_50_epoc']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

(1855, 22)
(1855, 22)


# Calculating similarity scores using Add New Layer BERT 30 EPOC

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer, models

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
class CustomModel(nn.Module):
    def __init__(self,sentence_bert):
        self.device='cuda'
        super(CustomModel, self).__init__()
        self.tokenize1 = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer = sentence_bert[0].auto_model.to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
        """self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )"""
        self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),  # ReLU activation function
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )
        # Freeze all the parameters in the BERT model
        for param in self.transformer.parameters():
            param.requires_grad = False
        import torch.nn.init as init
        # Initialize weights using He initialization suitable for ReLU activations
        """for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)"""
        # Initialize weights using He initialization suitable for ReLU activations
        for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                # He initialization for weights
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                # Zero initialization for biases (if applicable)
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)
   # def tokenize(self, text):
    #    # Implement your tokenization logic
     #   return self.tokenize1(text,return_tensors='pt')

    def tokenize(self, text):
        return self.tokenize1(text, return_tensors='pt', padding=True, truncation=True, max_length=512)


    def forward(self, tok_text):
        #tok_text = self.tokenizer(tok_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        #print('&'*10,tok_text['input_ids'],type(tok_text['input_ids']))
        outputs = self.transformer(**tok_text)#input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        custom_output = self.custom_layers(sequence_output)

        pool_input = {
        'token_embeddings': custom_output,  # Assuming this is [batch_size, seq_len, hidden_size]
        'attention_mask': tok_text['attention_mask']
        }

        return pool_input
sentence_bert = SentenceTransformer('bert-base-nli-mean-tokens')

# Create an instance of your custom model
custom_model = CustomModel(sentence_bert)

# Since SentenceTransformer expects a model that outputs a single embedding vector per input,
# you should define a pooling layer if the output is a sequence of token embeddings.
pooling_layer = models.Pooling(custom_model.transformer.config.hidden_size)
pooling_layer = sentence_bert[1].to('cuda')

# Create a Sentence Transformer model with custom processing
model = SentenceTransformer(modules=[custom_model, pooling_layer])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
import torch

#Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
#Bert_Fined_Tuned_Model = torch.load('/content/drive/MyDrive/Delete/Jeevan_cv_run1_add_new_layer_bert_1_EPOC_for_testing_only.pt')
#Bert_Fined_Tuned_Model.load_state_dict(torch.load('/content/drive/MyDrive/Delete/Jeevan_cv_run1_add_new_layer_bert_1_EPOC_for_testing_only.pt'))

model.load_state_dict(torch.load('folder/Jeevan_cv_run1_add_new_layer_bert_30_EPOC.pt'))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Add_New_Layer_Model_30_EPOC']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Add_New_Layer_Model_30_EPOC']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

del model

(1855, 23)
(1855, 23)


# Calculating similarity scores using Add New Layer BERT 50 EPOC

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer, models

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
class CustomModel(nn.Module):
    def __init__(self,sentence_bert):
        self.device='cuda'
        super(CustomModel, self).__init__()
        self.tokenize1 = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer = sentence_bert[0].auto_model.to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
        """self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )"""
        self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),  # ReLU activation function
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )
        # Freeze all the parameters in the BERT model
        for param in self.transformer.parameters():
            param.requires_grad = False
        import torch.nn.init as init
        # Initialize weights using He initialization suitable for ReLU activations
        """for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)"""
        # Initialize weights using He initialization suitable for ReLU activations
        for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                # He initialization for weights
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                # Zero initialization for biases (if applicable)
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)
   # def tokenize(self, text):
    #    # Implement your tokenization logic
     #   return self.tokenize1(text,return_tensors='pt')

    def tokenize(self, text):
        return self.tokenize1(text, return_tensors='pt', padding=True, truncation=True, max_length=512)


    def forward(self, tok_text):
        #tok_text = self.tokenizer(tok_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        #print('&'*10,tok_text['input_ids'],type(tok_text['input_ids']))
        outputs = self.transformer(**tok_text)#input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        custom_output = self.custom_layers(sequence_output)

        pool_input = {
        'token_embeddings': custom_output,  # Assuming this is [batch_size, seq_len, hidden_size]
        'attention_mask': tok_text['attention_mask']
        }

        return pool_input
sentence_bert = SentenceTransformer('bert-base-nli-mean-tokens')

# Create an instance of your custom model
custom_model = CustomModel(sentence_bert)

# Since SentenceTransformer expects a model that outputs a single embedding vector per input,
# you should define a pooling layer if the output is a sequence of token embeddings.
pooling_layer = models.Pooling(custom_model.transformer.config.hidden_size)
pooling_layer = sentence_bert[1].to('cuda')

# Create a Sentence Transformer model with custom processing
model = SentenceTransformer(modules=[custom_model, pooling_layer])

from transformers import AutoModel
from sentence_transformers import SentenceTransformer
import torch

#Bert_Fined_Tuned_Model = SentenceTransformer('bert-base-nli-mean-tokens')
#Bert_Fined_Tuned_Model = torch.load('/content/drive/MyDrive/Delete/Jeevan_cv_run1_add_new_layer_bert_1_EPOC_for_testing_only.pt')
#Bert_Fined_Tuned_Model.load_state_dict(torch.load('/content/drive/MyDrive/Delete/Jeevan_cv_run1_add_new_layer_bert_1_EPOC_for_testing_only.pt'))

model.load_state_dict(torch.load('folder/Jeevan_cv_run1_add_new_layer_bert_50_EPOC.pt'))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Add_New_Layer_Model_50_EPOC']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Add_New_Layer_Model_50_EPOC']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

del model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(1855, 24)
(1855, 24)


# Calculating similarity scores using Add New Layer BERT 100 EPOC

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer, models

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
class CustomModel(nn.Module):
    def __init__(self,sentence_bert):
        self.device='cuda'
        super(CustomModel, self).__init__()
        self.tokenize1 = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer = sentence_bert[0].auto_model.to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_tensors='pt')
        """self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )"""
        self.custom_layers = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),  # ReLU activation function
            nn.LayerNorm(768),
            nn.Dropout(0.1)
        )
        # Freeze all the parameters in the BERT model
        for param in self.transformer.parameters():
            param.requires_grad = False
        import torch.nn.init as init
        # Initialize weights using He initialization suitable for ReLU activations
        """for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)"""
        # Initialize weights using He initialization suitable for ReLU activations
        for layer in self.custom_layers:
            if isinstance(layer, nn.Linear):
                # He initialization for weights
                init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                # Zero initialization for biases (if applicable)
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)
   # def tokenize(self, text):
    #    # Implement your tokenization logic
     #   return self.tokenize1(text,return_tensors='pt')

    def tokenize(self, text):
        return self.tokenize1(text, return_tensors='pt', padding=True, truncation=True, max_length=512)


    def forward(self, tok_text):
        #tok_text = self.tokenizer(tok_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        #print('&'*10,tok_text['input_ids'],type(tok_text['input_ids']))
        outputs = self.transformer(**tok_text)#input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        custom_output = self.custom_layers(sequence_output)

        pool_input = {
        'token_embeddings': custom_output,  # Assuming this is [batch_size, seq_len, hidden_size]
        'attention_mask': tok_text['attention_mask']
        }

        return pool_input
sentence_bert = SentenceTransformer('bert-base-nli-mean-tokens')

# Create an instance of your custom model
custom_model = CustomModel(sentence_bert)

# Since SentenceTransformer expects a model that outputs a single embedding vector per input,
# you should define a pooling layer if the output is a sequence of token embeddings.
pooling_layer = models.Pooling(custom_model.transformer.config.hidden_size)
pooling_layer = sentence_bert[1].to('cuda')

# Create a Sentence Transformer model with custom processing
model = SentenceTransformer(modules=[custom_model, pooling_layer])

from transformers import AutoModel
from sentence_transformers import SentenceTransformer
import torch


model.load_state_dict(torch.load('folder/Jeevan_cv_run1_add_new_layer_bert_100_EPOC.pt'))


from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np


print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Add_New_Layer_Model_100_EPOC']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Add_New_Layer_Model_100_EPOC']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

del model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(1855, 25)
(1855, 25)


In [ ]:
sample_data.head()

title_Recipe1       title_Recipe2  Similarity title_Recipe1_mod  \
0  ""Chicken Stew""  ""Angel Biscuits""        0.05      Chicken Stew   
1  ""Chicken Stew""      ""Beer Bread""        0.05      Chicken Stew   
2  ""Chicken Stew""   ""Chicken Divan""        0.85      Chicken Stew   
3  ""Chicken Stew""     ""Chicken Ole""        0.80      Chicken Stew   
4  ""Chicken Stew""    ""Chicken Stew""        1.00      Chicken Stew   

  title_Recipe2_mod  tfidf_similarity  Bert_similarity         title_x  \
0    Angel Biscuits          0.024506         0.718295  Angel Biscuits   
1        Beer Bread          0.048481         0.753868      Beer Bread   
2     Chicken Divan          0.043482         0.913871   Chicken Divan   
3       Chicken Ole          0.105818         0.895192     Chicken Ole   
4      Chicken Stew          1.000000         1.000000    Chicken Stew   

                                   Recipe1_sentences       title_y  ...  \
0  Angel Biscuits ["5 c. flour", "3 Tbsp. sugar",...  Chicken Stew  ...   
1  Beer Bread ["3 c. self rising flour", "1 - 12 ...  Chicken Stew  ...   
2  Chicken Divan ["1/4 c. margarine", "1/4 c. cho...  Chicken Stew  ...   
3  Chicken Ole ["4 chicken breasts, cooked", "1 c...  Chicken Stew  ...   
4  Chicken Stew ["3 lb. chicken, boiled", "4 medi...  Chicken Stew  ...   

  Robert_Similarity_5_epoc  Robert_Similarity_10_epoc  \
0                 0.123343                   0.136814   
1                 0.108734                   0.121350   
2                 0.814694                   0.829597   
3                 0.872819                   0.861640   
4                 1.000000                   1.000000   

   Robert_Similarity_20_epoc  Robert_Similarity_30_epoc  \
0                   0.133249                   0.132106   
1                   0.103526                   0.083370   
2                   0.850317                   0.838664   
3                   0.860606                   0.848837   
4                   1.000000                   1.000000   

   RecipeBert_finetuning_v3  Fine_Tuned_Bert_Similarity_40_epoc  \
0                  0.138439                            0.155239   
1                  0.099758                            0.082167   
2                  0.877858                            0.851222   
3                  0.901725                            0.845041   
4                  1.000000                            1.000000   

   Robert_Similarity_50_epoc  Add_New_Layer_Model_30_EPOC  \
0                   0.157055                     0.131796   
1                   0.073003                     0.105008   
2                   0.852554                     0.680780   
3                   0.843494                     0.769090   
4                   1.000000                     1.000000   

   Add_New_Layer_Model_50_EPOC  Add_New_Layer_Model_100_EPOC  
0                     0.139907                      0.147049  
1                     0.080925                      0.053674  
2                     0.676324                      0.672926  
3                     0.794044                      0.810611  
4                     1.000000                      1.000000  

[5 rows x 26 columns]

# Calculating similarity scores using ROBERTA base

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

Robert_Model = SentenceTransformer('distilbert/distilroberta-base')

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np

# # Generate embeddings for the test sentences
# embeddings1 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe1_sentences'].tolist(), convert_to_tensor=True)
# embeddings2 = Bert_Fined_Tuned_Model.encode(sample_data['Recipe2_sentences'].tolist(), convert_to_tensor=True)

print(sample_data.shape)
sample_data=sample_data.dropna()
print(sample_data.shape)

from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Assuming 'data' is your DataFrame containing the custom dataset
# Generate embeddings for the test sentences
list_recipe1=sample_data['Recipe1_sentences'].drop_duplicates().tolist()
list_recipe2=sample_data['Recipe2_sentences'].drop_duplicates().tolist()

embeddings1_unique = Robert_Model.encode(list_recipe1, convert_to_tensor=True)
embeddings2_unique = Robert_Model.encode(list_recipe2, convert_to_tensor=True)
sample_data['Robert_base_Similarity']=None


from sentence_transformers import SentenceTransformer, util
def CosineSim(title_Recipe1,title_Recipe2):
    #x=cosine_similarity(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)])

    return util.pytorch_cos_sim(embeddings1_unique[list_recipe1.index(title_Recipe1)],embeddings2_unique[list_recipe2.index(title_Recipe2)]).item()


sample_data['Robert_base_Similarity']=sample_data.apply(lambda row: CosineSim(row['Recipe1_sentences'],row['Recipe2_sentences']),axis=1)

sample_data.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(1855, 26)
(1855, 26)


title_Recipe1       title_Recipe2  Similarity title_Recipe1_mod  \
0  ""Chicken Stew""  ""Angel Biscuits""        0.05      Chicken Stew   
1  ""Chicken Stew""      ""Beer Bread""        0.05      Chicken Stew   
2  ""Chicken Stew""   ""Chicken Divan""        0.85      Chicken Stew   
3  ""Chicken Stew""     ""Chicken Ole""        0.80      Chicken Stew   
4  ""Chicken Stew""    ""Chicken Stew""        1.00      Chicken Stew   

  title_Recipe2_mod  tfidf_similarity  Bert_similarity         title_x  \
0    Angel Biscuits          0.024506         0.718295  Angel Biscuits   
1        Beer Bread          0.048481         0.753868      Beer Bread   
2     Chicken Divan          0.043482         0.913871   Chicken Divan   
3       Chicken Ole          0.105818         0.895192     Chicken Ole   
4      Chicken Stew          1.000000         1.000000    Chicken Stew   

                                   Recipe1_sentences       title_y  ...  \
0  Angel Biscuits ["5 c. flour", "3 Tbsp. sugar",...  Chicken Stew  ...   
1  Beer Bread ["3 c. self rising flour", "1 - 12 ...  Chicken Stew  ...   
2  Chicken Divan ["1/4 c. margarine", "1/4 c. cho...  Chicken Stew  ...   
3  Chicken Ole ["4 chicken breasts, cooked", "1 c...  Chicken Stew  ...   
4  Chicken Stew ["3 lb. chicken, boiled", "4 medi...  Chicken Stew  ...   

  Robert_Similarity_10_epoc  Robert_Similarity_20_epoc  \
0                  0.136814                   0.133249   
1                  0.121350                   0.103526   
2                  0.829597                   0.850317   
3                  0.861640                   0.860606   
4                  1.000000                   1.000000   

   Robert_Similarity_30_epoc  RecipeBert_finetuning_v3  \
0                   0.132106                  0.138439   
1                   0.083370                  0.099758   
2                   0.838664                  0.877858   
3                   0.848837                  0.901725   
4                   1.000000                  1.000000   

   Fine_Tuned_Bert_Similarity_40_epoc  Robert_Similarity_50_epoc  \
0                            0.155239                   0.157055   
1                            0.082167                   0.073003   
2                            0.851222                   0.852554   
3                            0.845041                   0.843494   
4                            1.000000                   1.000000   

   Add_New_Layer_Model_30_EPOC  Add_New_Layer_Model_50_EPOC  \
0                     0.131796                     0.139907   
1                     0.105008                     0.080925   
2                     0.680780                     0.676324   
3                     0.769090                     0.794044   
4                     1.000000                     1.000000   

   Add_New_Layer_Model_100_EPOC  Robert_base_Similarity  
0                      0.147049                0.990071  
1                      0.053674                0.977432  
2                      0.672926                0.987982  
3                      0.810611                0.990325  
4                      1.000000                1.000000  

[5 rows x 27 columns]

Saving CSV file to Google drive. Will be using this file for calculating Correlation and NDCG Metrics in R

In [ ]:
sample_data.to_csv('folder/jeevan_cv_run1_all_models_combined_validation_data_output_v2.csv', index = False)

#References:
1] fine tuning - https://colab.research.google.com/github/huggingface/blog/blob/main/notebooks/95_Training_Sentence_Transformers.ipynb
2]model development from scratch -https://huggingface.co/blog/how-to-train-sentence-transformers